# A/B Testing Reference Guide
## Quick Reference for Concepts, Techniques, and Interview Prep

---

## 🎯 How to Use This Guide

This notebook serves as a **quick reference** for all concepts covered in the masterclass series. Use it to:

1. **Review before interviews** - Refresh key concepts and decision frameworks
2. **Look up techniques** - Find when to use what method
3. **Check code patterns** - Copy-paste templates for common tasks
4. **Understand trade-offs** - Quick comparison tables

### The A/B Testing Lifecycle (Your Mental Model)

In interviews, always anchor your answers to this lifecycle:

```
┌─────────────────────────────────────────────────────────────────┐
│                    THE EXPERIMENTATION LIFECYCLE                │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  1. FRAME THE QUESTION                                          │
│     • What business problem are we solving?                     │
│     • What's our hypothesis (with mechanism and risks)?         │
│                                                                 │
│  2. DESIGN THE EXPERIMENT                                       │
│     • Choose metrics (primary + guardrails)                     │
│     • Calculate sample size (power analysis)                    │
│     • Define success criteria upfront                           │
│                                                                 │
│  3. VALIDATE THE DATA                                           │
│     • Data quality checks                                       │
│     • SRM check (did randomization work?)                       │
│     • Classify: RCT vs. observational                           │
│                                                                 │
│  4. ANALYZE RESULTS                                             │
│     • Primary metric test                                       │
│     • Variance reduction (CUPED/CUPAC)                          │
│     • Multiple testing correction                               │
│     • Heterogeneous effects (if relevant)                       │
│                                                                 │
│  5. INTERPRET & DECIDE                                          │
│     • Guardrail evaluation                                      │
│     • Novelty effect check                                      │
│     • Business impact calculation                               │
│     • Ship / Hold / Abandon decision                            │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

---

## 1. Dataset Classification

### When to Use Which Classification

| Allocation | Classification | SRM Action | Causal Claims |
|------------|----------------|------------|---------------|
| 40-60% | **RCT** | Compare to expected ratio | Full causal inference |
| 10-40% or 60-90% | **Designed Imbalance** | Compare to intended ratio | Full causal inference |
| <10% or >90% | **Observational** | Note, don't gate | Associational only |

### Code Template

In [ ]:
def classify_dataset(treatment_ratio):
    """Classify dataset based on treatment allocation."""
    if 0.4 <= treatment_ratio <= 0.6:
        return 'RCT', 'Compare to expected ratio'
    elif 0.1 <= treatment_ratio <= 0.9:
        return 'DESIGNED_IMBALANCE', 'Compare to intended ratio'
    else:
        return 'OBSERVATIONAL', 'Causal claims limited'

# Usage:
# data_type, action = classify_dataset(treatment_count / total_count)

---

## 2. SRM (Sample Ratio Mismatch)

### Two-Stage Gating Framework

```
┌─────────────────────────────────────────────────────────────┐
│                    TWO-STAGE SRM GATING                     │
├─────────────────────────────────────────────────────────────┤
│                                                             │
│  Stage 1: STATISTICAL SIGNIFICANCE                          │
│     • Chi-square test: p < 0.01?                            │
│     • If NO → Pass, proceed to analysis                     │
│     • If YES → Check Stage 2                                │
│                                                             │
│  Stage 2: PRACTICAL SIGNIFICANCE                            │
│     • Deviation > 1 percentage point?                       │
│     • If NO → Warning only, proceed with caution            │
│     • If YES → HARD GATE, stop analysis                     │
│                                                             │
└─────────────────────────────────────────────────────────────┘
```

### Code Template

In [ ]:
from ab_testing.core import randomization

def check_srm_with_gating(n_control, n_treatment, expected_ratio=[0.5, 0.5], is_rct=True):
    """Two-stage SRM check with gating logic."""
    
    srm_result = randomization.srm_check(
        n_control=n_control,
        n_treatment=n_treatment,
        expected_ratio=expected_ratio,
        alpha=0.01
    )
    
    # Stage 1: Statistical significance
    stat_sig = srm_result['srm_detected']
    
    # Stage 2: Practical significance (>1 percentage point)
    deviation = srm_result['max_pp_deviation']
    practical_sig = srm_result['practical_significant']
    
    # Decision logic
    if is_rct and srm_result['srm_severe']:
        return 'HARD_GATE', 'Stop analysis, investigate randomization'
    elif srm_result['srm_warning']:
        return 'WARNING', 'Statistical but not practical significance, proceed with caution'
    else:
        return 'PASS', 'Randomization appears correct'

# Usage:
# status, message = check_srm_with_gating(control_n, treatment_n, expected_ratio=[0.5, 0.5])

### 💡 Interview Question: SRM Interpretation

**Q**: *"The SRM check is statistically significant but the deviation is only 0.3%. Should we stop?"*

**A**: *"No. With large samples, even tiny deviations become statistically significant. A 0.3% deviation (49.7% vs 50.3%) won't meaningfully bias our treatment effect estimates. We'd only stop if the deviation were large enough to indicate a systematic problem—typically >1 percentage point. This is statistical but not practical significance."*

---

## 3. Statistical Tests

### When to Use Which Test

| Outcome Type | Metric Example | Test | Module Function |
|--------------|----------------|------|------------------|
| Binary | Conversion, Retention | Z-test for proportions | `frequentist.z_test_proportions()` |
| Continuous | Revenue, Time on site | T-test or Z-test | `frequentist.t_test()` |
| Ratio | Revenue/user, CTR | Delta method | `ratio_metrics.ratio_metric_test()` |
| Count | Purchases, Clicks | Poisson or NB | Depends on overdispersion |

### Code Templates

In [ ]:
from ab_testing.core import frequentist
from ab_testing.advanced import ratio_metrics

# Binary outcome (conversion, retention)
def test_binary_metric(control, treatment, alpha=0.05):
    """Z-test for binary outcomes."""
    return frequentist.z_test_proportions(
        control=control,
        treatment=treatment,
        alpha=alpha
    )

# Ratio metric (revenue per user, clicks per session)
def test_ratio_metric(control_num, control_denom, treatment_num, treatment_denom, alpha=0.05):
    """Delta method for ratio metrics."""
    return ratio_metrics.ratio_metric_test(
        control_num=control_num,
        control_denom=control_denom,
        treatment_num=treatment_num,
        treatment_denom=treatment_denom,
        alpha=alpha
    )

---

## 4. Multiple Testing Correction

### When to Use Which Method

| Method | Controls | When to Use |
|--------|----------|-------------|
| **None** | - | Single metric |
| **Bonferroni** | FWER | Very conservative; high cost of false positives |
| **Benjamini-Hochberg** | FDR | Standard choice for 2-20 metrics |
| **Holm** | FWER | Less conservative than Bonferroni |

### Decision Framework

- **1 metric**: No correction needed
- **2-5 metrics**: Benjamini-Hochberg (FDR control)
- **6-20 metrics**: Benjamini-Hochberg
- **20+ metrics**: Consider grouping or hierarchical testing

### Code Template

In [ ]:
from ab_testing.advanced import multiple_testing

def correct_multiple_tests(p_values, method='benjamini-hochberg', alpha=0.05):
    """Apply multiple testing correction."""
    if method == 'benjamini-hochberg':
        return multiple_testing.benjamini_hochberg(p_values=p_values, alpha=alpha)
    elif method == 'bonferroni':
        adjusted = [min(p * len(p_values), 1.0) for p in p_values]
        return {'adjusted_p_values': adjusted, 'reject_null': [p < alpha for p in adjusted]}
    else:
        raise ValueError(f"Unknown method: {method}")

# Usage:
# result = correct_multiple_tests([0.03, 0.08, 0.001], method='benjamini-hochberg')

### 💡 Interview Question: Multiple Testing

**Q**: *"We're testing 5 metrics. Do we need to correct for multiple testing?"*

**A**: *"Yes. Testing 5 metrics at α=0.05 gives us a 1-(0.95)^5 ≈ 23% chance of at least one false positive. I'd use Benjamini-Hochberg to control the False Discovery Rate—it's less conservative than Bonferroni while still protecting against inflated error rates. The key distinction: BH controls the expected proportion of false discoveries among rejections, while Bonferroni controls the probability of any false positive."*

---

## 5. Variance Reduction

### CUPED vs. CUPAC Comparison

| Aspect | CUPED | CUPAC |
|--------|-------|-------|
| **Adjustment** | Linear | ML model |
| **Covariates** | Single (pre-experiment metric) | Multiple |
| **Complexity** | Low | High |
| **Data requirement** | Small OK | Large needed |
| **Variance reduction** | ~30% typical | ~40%+ typical |
| **When to use** | Default choice | Rich covariates, large samples |

### Code Templates

In [ ]:
from ab_testing.variance_reduction import cuped, cupac

# CUPED (linear adjustment with pre-experiment metric)
def run_cuped(y, treatment, x_pre, alpha=0.05):
    """CUPED variance reduction."""
    return cuped.cuped_ab_test(
        y=y,
        treatment=treatment,
        x_pre=x_pre,
        alpha=alpha
    )

# CUPAC (ML-enhanced variance reduction)
def run_cupac(y, treatment, X, alpha=0.05):
    """CUPAC variance reduction with ML."""
    return cupac.cupac_ab_test(
        y=y,
        treatment=treatment,
        X=X,
        alpha=alpha
    )

### 💡 Interview Question: Variance Reduction

**Q**: *"When would you use CUPAC instead of CUPED?"*

**A**: *"CUPAC shines when you have large samples (100K+) and rich covariate data with non-linear relationships. For example, at Netflix, they use CUPAC because they have billions of user features. CUPED is simpler and works well with just a pre-experiment measure of the outcome. If I have limited features or smaller samples, CUPED is usually sufficient and more interpretable. The key is whether the added complexity of ML provides meaningful variance reduction."*

---

## 6. Heterogeneous Treatment Effects (HTE)

### When to Analyze HTE

| Scenario | Analyze HTE? | Why |
|----------|--------------|-----|
| Personalization decision | Yes | Need to know who benefits |
| Targeting decision | Yes | Optimize ad spend |
| Simple ship/no-ship | Maybe | If concerned about harm to segments |
| Small sample | No | Insufficient power for subgroups |

### Code Template

In [ ]:
from ab_testing.advanced import hte
import numpy as np

def analyze_heterogeneity(X, treatment, y):
    """Analyze heterogeneous treatment effects using X-Learner."""
    
    # Fit X-Learner
    xlearner = hte.XLearner()
    xlearner.fit(X=X, treatment=treatment, y=y)
    
    # Get individual treatment effects
    cate = xlearner.predict(X)
    
    # Summary statistics
    return {
        'mean_cate': np.mean(cate),
        'std_cate': np.std(cate),
        'pct_positive': (cate > 0).mean(),
        'pct_negative': (cate < 0).mean(),
        'cate_estimates': cate
    }

### 💡 Interview Question: HTE Interpretation

**Q**: *"The ATE is +5%, but 30% of users show negative effects. What do you recommend?"*

**A**: *"This is a judgment call with no single correct answer. I'd consider: (1) Can we reliably identify who benefits vs. who's harmed? (2) What's the cost of targeting vs. broad rollout? (3) How severe is the harm to the 30%? If we can target accurately and it's operationally feasible, I'd recommend targeting only positive responders. If targeting is impractical, we need to weigh aggregate benefit against harm to a minority. I'd also investigate WHY some users are harmed—there might be a fixable product issue."*

---

## 7. Sequential Testing

### When to Use Sequential Testing

| Scenario | Use Sequential? | Boundary |
|----------|-----------------|----------|
| Long experiment, want early stopping | Yes | O'Brien-Fleming |
| High-stakes, need flexibility | Yes | Group sequential |
| Short experiment, will wait | No | Fixed sample |
| Need maximum power | No | Fixed sample |

### Code Template

In [ ]:
from ab_testing.advanced import sequential

def get_sequential_boundaries(n_looks=5, alpha=0.05):
    """Generate O'Brien-Fleming boundaries for sequential testing."""
    return sequential.obrien_fleming_boundaries(
        n_looks=n_looks,
        alpha=alpha,
        one_sided=False
    )

def check_sequential_decision(z_statistic, look_number, boundaries):
    """Check if we can stop at current interim analysis."""
    boundary = boundaries['z_boundaries'][look_number - 1]
    
    if abs(z_statistic) > boundary:
        if z_statistic > 0:
            return 'STOP_POSITIVE', f'Z={z_statistic:.2f} > {boundary:.2f}'
        else:
            return 'STOP_NEGATIVE', f'Z={z_statistic:.2f} < -{boundary:.2f}'
    else:
        return 'CONTINUE', f'Z={z_statistic:.2f} within ±{boundary:.2f}'

---

## 8. Guardrails

### Guardrail Threshold Guidelines

| Metric Type | Typical Tolerance | Rationale |
|-------------|-------------------|------------|
| Revenue | -2% to -5% | High business impact |
| Retention | -1% to -2% | Leading indicator |
| Engagement | -5% to -10% | More tolerant |
| Load time | +10% to +20% | Technical |

### Code Template

In [ ]:
from ab_testing.diagnostics import guardrails

def check_guardrail(control, treatment, metric_name, delta, metric_type='relative'):
    """Run non-inferiority test for guardrail metric."""
    result = guardrails.non_inferiority_test(
        control=control,
        treatment=treatment,
        delta=delta,  # e.g., -0.02 for 2% max degradation
        metric_type=metric_type,
        alpha=0.05
    )
    result['metric_name'] = metric_name
    return result

---

## 9. Decision Framework

### Ship / Hold / Abandon Logic

```
┌─────────────────────────────────────────────────────────────────┐
│                     DECISION FRAMEWORK                          │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  ✅ SHIP if:                                                    │
│     • Primary metric significant AND positive                   │
│     • ALL guardrails pass                                       │
│     • No severe novelty effect (or acceptable)                  │
│     • Business impact justifies implementation                  │
│                                                                 │
│  ❌ ABANDON if:                                                 │
│     • Primary metric significant AND negative                   │
│     • ANY guardrail fails                                       │
│     • Severe SRM in RCT                                         │
│                                                                 │
│  ⚪ HOLD if:                                                    │
│     • Primary metric not significant                            │
│     • Mixed signals (some positive, some concerning)            │
│     • Novelty detected, need holdout to verify                  │
│     • Observational data, need proper RCT                       │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

### Code Template

In [ ]:
def make_decision(primary_result, guardrail_results, data_type='RCT'):
    """Make ship/hold/abandon decision."""
    
    primary_sig = primary_result.get('significant', False)
    primary_positive = primary_result.get('relative_lift', 0) > 0
    all_guardrails_pass = all(g.get('passed', True) for g in guardrail_results)
    
    # Observational data gets special treatment
    if data_type == 'OBSERVATIONAL':
        return 'HOLD', 'Observational data - causal claims limited, consider RCT'
    
    # Decision logic
    if primary_sig and not primary_positive:
        return 'ABANDON', 'Primary metric significantly negative'
    elif not all_guardrails_pass:
        return 'ABANDON', 'One or more guardrails failed'
    elif primary_sig and primary_positive and all_guardrails_pass:
        return 'SHIP', 'Primary positive, all guardrails pass'
    else:
        return 'HOLD', 'Primary not significant or mixed signals'

---

## 10. Business Impact Formulas

### Key Calculations

| Metric | Formula |
|--------|----------|
| Monthly impact | `visitors × lift × value_per_conversion` |
| Annual impact | `monthly_impact × 12` |
| ROI | `(annual_impact - cost) / cost` |
| Break-even lift | `cost / (visitors × value × 12)` |
| NNT | `1 / absolute_lift` |

### Code Template

In [ ]:
def calculate_business_impact(baseline_rate, lift, monthly_visitors, value_per_conversion, implementation_cost):
    """Calculate business impact metrics."""
    
    # Current state
    current_conversions = monthly_visitors * baseline_rate
    current_revenue = current_conversions * value_per_conversion
    
    # New state
    new_rate = baseline_rate * (1 + lift)
    new_conversions = monthly_visitors * new_rate
    new_revenue = new_conversions * value_per_conversion
    
    # Impact
    monthly_impact = new_revenue - current_revenue
    annual_impact = monthly_impact * 12
    roi = (annual_impact - implementation_cost) / implementation_cost if implementation_cost > 0 else float('inf')
    
    return {
        'monthly_impact': monthly_impact,
        'annual_impact': annual_impact,
        'roi': roi,
        'additional_conversions_monthly': new_conversions - current_conversions
    }

---

## 11. Module Reference

### Quick Lookup Table

| Task | Module | Function |
|------|--------|----------|
| SRM check | `ab_testing.core.randomization` | `srm_check()` |
| Power analysis | `ab_testing.core.power` | `power_analysis_summary()` |
| Z-test (proportions) | `ab_testing.core.frequentist` | `z_test_proportions()` |
| T-test | `ab_testing.core.frequentist` | `t_test()` |
| CUPED | `ab_testing.variance_reduction.cuped` | `cuped_ab_test()` |
| CUPAC | `ab_testing.variance_reduction.cupac` | `cupac_ab_test()` |
| Multiple testing | `ab_testing.advanced.multiple_testing` | `benjamini_hochberg()` |
| Ratio metrics | `ab_testing.advanced.ratio_metrics` | `ratio_metric_test()` |
| HTE (X-Learner) | `ab_testing.advanced.hte` | `XLearner` |
| Sequential testing | `ab_testing.advanced.sequential` | `obrien_fleming_boundaries()` |
| Guardrails | `ab_testing.diagnostics.guardrails` | `non_inferiority_test()` |
| Decision | `ab_testing.diagnostics.guardrails` | `evaluate_guardrails()` |

---

## 12. Common Interview Questions

### Quick Answers

| Question | Key Points |
|----------|------------|
| *"Walk me through how you'd design an A/B test"* | Frame hypothesis → Choose metrics → Power analysis → Define success criteria → Run → Validate → Analyze → Decide |
| *"What's the difference between statistical and practical significance?"* | Statistical: unlikely due to chance. Practical: large enough to matter for business. Need both for action. |
| *"How do you handle multiple testing?"* | BH for FDR control with 2-20 metrics. Bonferroni if false positives very costly. None for single metric. |
| *"What would make you not trust an A/B test result?"* | SRM failure, data quality issues, novelty effects, observational data, underpowered test. |
| *"The test isn't significant. What do you conclude?"* | Check power/MDE first. Not significant ≠ no effect. Could lack power to detect. |
| *"Stakeholders want to peek at results daily. Is that okay?"* | Not with fixed-sample tests (inflates error). Use sequential testing with O'Brien-Fleming boundaries. |
| *"How do you translate results to business impact?"* | Convert lift to revenue using visitors × lift × value. Include CI for uncertainty range. |

---

## 13. Industry Best Practices Summary

### By Company

| Company | Notable Practice |
|---------|------------------|
| **Microsoft** | Automated SRM checks, blocks analysis on failure |
| **Netflix** | CUPAC standard, 40%+ variance reduction |
| **Booking.com** | Stricter SRM alpha (0.001), conservative approach |
| **Uber** | X-Learner for personalized pricing |
| **Spotify** | CUPED as baseline, ~30% average reduction |
| **LinkedIn** | O'Brien-Fleming for sequential testing |
| **Zynga/King** | 2-4 week minimum for game changes |

---

## Key Takeaways: The Mental Model

### What Separates Strong Candidates

1. **Lead with the lifecycle, not the math.**
   - Start with framing and metrics, not formulas.

2. **Validate before you analyze.**
   - Data quality and SRM come first.

3. **Articulate trade-offs.**
   - Every decision has pros and cons. Name them.

4. **Connect to business impact.**
   - P-values don't pay salaries—revenue does.

5. **Acknowledge uncertainty.**
   - Use confidence intervals. Know what you don't know.

6. **Know when to use advanced techniques.**
   - CUPAC, HTE, sequential testing are tools, not goals.

7. **Judgment > formulas.**
   - The hardest part isn't the math—it's the decision.

---

**This is your foundation. The notebooks provide the depth. Use both together.**